In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install transformers

In [ ]:
!pip install --pre deepchem
import deepchem
deepchem.__version__

In [ ]:
import sys
!test -d bertviz_repo && echo "FYI: bertviz_repo directory already exists, to pull latest version uncomment this line: !rm -r bertviz_repo"
# !rm -r bertviz_repo # Uncomment if you need a clean pull from repo
!test -d bertviz_repo || git clone https://github.com/jessevig/bertviz bertviz_repo
if not 'bertviz_repo' in sys.path:
  sys.path += ['bertviz_repo']
!pip install regex

In [ ]:
import pandas as pd
# Read the datasets
train = pd.read_csv('/content/drive/Shareddrives/1:1 Awani Gadre/Dataset/train.csv')
test = pd.read_csv('/content/drive/Shareddrives/1:1 Awani Gadre/Dataset/test.csv')

In [ ]:
from transformers import AutoModelForMaskedLM, AutoTokenizer, RobertaModel, RobertaTokenizer
import pandas as pd
import torch

X_train = train['canonical_smiles']
y_train = train['pIC50']
X_test = test['canonical_smiles']
y_test = test['pIC50']


# Function to get embeddings
def get_embeddings(data):
    token_ids = [tokenizer.encode(smile, add_special_tokens=True, max_length=512, truncation=True) for smile in data]
    embeddings = []

    with torch.no_grad():
        for ids in token_ids:
            ids_tensor = torch.tensor(ids).unsqueeze(0)
            output = model(ids_tensor)[0]
            avg_embedding = output[0].mean(dim=0)
            embeddings.append(avg_embedding.numpy())

    return embeddings

# Load model and tokenizer
model = RobertaModel.from_pretrained("seyonec/ChemBERTa-zinc-base-v1")
tokenizer = AutoTokenizer.from_pretrained("seyonec/ChemBERTa-zinc-base-v1")

# Get embeddings
train_embeddings = get_embeddings(X_train)
test_embeddings = get_embeddings(X_test)

# Combine embeddings and descriptors
train_embed_df = pd.concat([pd.DataFrame(train_embeddings), y_train], axis=1)
test_embed_df = pd.concat([pd.DataFrame(test_embeddings), y_test], axis=1)



# Optionally, save combined data to CSV
train_embed_df.to_csv('/content/drive/Shareddrives/1:1 Awani Gadre/Dataset/train_chemberta_descriptors_zinc.csv', index=False)
test_embed_df.to_csv('/content/drive/Shareddrives/1:1 Awani Gadre/Dataset/test_chemberta_descriptors_zinc.csv', index=False)